In [1]:
# Cell 1: Imports and Setup
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_theme()

# Import our modules
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Cell 2: Import Custom Modules
from src.data import DataLoader, PortfolioConfig
from src.risk import RiskManager, RiskConfig
from src.monte_carlo import MonteCarlo, SimConfig
from src.regime import MarketRegimeDetector, RegimeConfig

In [3]:
# Cell 3: Configure and Load Data
# Configure Portfolio
portfolio_config = PortfolioConfig(
    tickers=['NVDA', 'AMD', 'INTC','ASML'],  # Semiconductor stocks
    start_date='2017-01-01',
    end_date='2024-01-01',
    weights=[0.4, 0.3, 0.15,0.15]
)

# Load Data
data_loader = DataLoader(portfolio_config)
market_data = data_loader.load_data()

# Quick data check
print("Data loaded successfully:")
print(f"Shape of returns: {market_data['returns'].shape}")
print(f"Date range: {market_data['returns'].index[0]} to {market_data['returns'].index[-1]}")

2024-12-01 18:40:31,524 - src.data - INFO - Connecting to Alpaca API (Paper Trading: True)
2024-12-01 18:40:31,525 - src.data - INFO - Loading market data...
2024-12-01 18:40:32,912 - src.data - INFO - Successfully loaded data for 4 tickers


Data loaded successfully:
Shape of returns: (1759, 4)
Date range: 2017-01-04 05:00:00+00:00 to 2023-12-29 05:00:00+00:00


In [4]:
# Cell 4: Risk Analysis
# Configure Risk Analysis
risk_config = RiskConfig(
    confidence_level=0.95,
    max_drawdown_limit=0.20,
    volatility_target=0.15,
    var_calculation_method="historical"
)

# Initialize Risk Manager and Calculate Metrics
risk_manager = RiskManager(risk_config)
# Calculate portfolio returns
portfolio_returns = market_data['returns'].mean(axis=1)
risk_metrics = risk_manager.calculate_metrics(portfolio_returns)

# Print key metrics
print("\nKey Risk Metrics:")
print(f"Portfolio Volatility: {risk_metrics['portfolio_volatility']:.2%}")
print(f"VaR (95%): {risk_metrics['var_95']:.2%}")
print(f"Expected Shortfall: {risk_metrics['expected_shortfall_95']:.2%}")
print(f"Sharpe Ratio: {risk_metrics['sharpe_ratio']:.2f}")


Key Risk Metrics:
Portfolio Volatility: 37.91%
VaR (95%): -3.81%
Expected Shortfall: -5.44%
Sharpe Ratio: 0.81


In [5]:
# Cell 5: Monte Carlo Simulation
# Configure and Run Monte Carlo
sim_config = SimConfig(
    n_sims=100000,
    n_days=252,
    distribution='t'  # Using Student's t for fat tails
)

monte_carlo = MonteCarlo(sim_config)
mc_results = monte_carlo.simulate(market_data)

print("Monte Carlo Simulation completed")
print(f"Number of simulations: {sim_config.n_sims}")
print(f"Forecast horizon: {sim_config.n_days} days")


Running Monte Carlo simulation...
Monte Carlo Simulation completed
Number of simulations: 100000
Forecast horizon: 252 days


In [6]:
# Cell 6: Regime Detection
# Configure and Run Regime Detection
regime_config = RegimeConfig(
    n_regimes=3,
    window_size=21
)

regime_detector = MarketRegimeDetector(regime_config)
regimes = regime_detector.fit_predict(market_data['returns'].mean(axis=1))

# Print regime statistics
print("Regime Distribution:")
print(regimes.value_counts(normalize=True))

Regime Distribution:
Low Vol       0.566231
Medium Vol    0.318931
High Vol      0.114838
dtype: float64
